window segmentation of edf file

In [ ]:
def read_edf(files_train):
    time_window = 8
    time_step = 4

    if os.path.exists('/kaggle/input/mit-chb-processed/signal_samples.npy')&os.path.exists('/kaggle/input/mit-chb-processed/is_sz.npy'):
        array_signals=np.load('/kaggle/input/mit-chb-processed/signal_samples.npy')
        array_is_sz=np.load('/kaggle/input/mit-chb-processed/is_sz.npy')
    else:
        p = 0.01
        counter = 0
        step_window = 0
        for temp_f in files_train:
            temp_edf =  mne.io.read_raw_edf(temp_f)
            temp_labels = temp_edf.ch_names
            if sum([any([0 if re.match(c, l) is None else 1 for l in temp_edf.ch_names]) for c in ch_labels])==len(ch_labels):
                time_window = 8
                time_step = 4
                fs = int(1/(temp_edf.times[1]-temp_edf.times[0]))
                step_window = time_window*fs
                step = time_step*fs

                temp_is_sz = np.zeros((temp_edf.n_times,))
                if os.path.exists(temp_f+'.seizures'):
                    temp_annotation = wfdb.rdann(temp_f, 'seizures')
                    for i in range(int(temp_annotation.sample.size/2)):
                        temp_is_sz[temp_annotation.sample[i*2]:temp_annotation.sample[i*2+1]]=1
                temp_len = temp_edf.n_times

                temp_is_sz_ind = np.array(
                    [temp_is_sz[i*step:i*step+step_window].sum()/step_window for i in range((temp_len-step_window)//step)]
                )

                temp_0_sample_size = round(p*np.where(temp_is_sz_ind==0)[0].size)
                temp_1_sample_size = np.where(temp_is_sz_ind>0)[0].size

                counter = counter + temp_0_sample_size + temp_1_sample_size
            temp_edf.close()
            return counter, ch_labels, step_window

def extract_edf2(files_train, counter, ch_labels, step_window):
        array_signals = np.zeros((counter, len(ch_labels), step_window), dtype=np.float32)
        array_is_sz = np.zeros(counter, dtype=bool)

        counter = 0
        for n, temp_f in enumerate(tqdm.tqdm(files_train)):
            to_log = 'No. {}: Reading. '.format(n)
            temp_edf =  mne.io.read_raw_edf(temp_f)
            temp_labels = temp_edf.ch_names
            n_label_match = sum([any([0 if re.match(c, l)==None else 1 for l in temp_edf.ch_names]) for c in ch_labels])
            if n_label_match==len(ch_labels):
                ch_mapping = {sorted([l for l in temp_edf.ch_names if re.match(c, l)!=None ])[0]:c for c in ch_labels}
                temp_edf.rename_channels(ch_mapping)
                #temp_edf = temp_edf.pick(ch_labels)

                temp_is_sz = np.zeros((temp_edf.n_times,))
                temp_signals = temp_edf.get_data(picks=ch_labels)*1e6

                if os.path.exists(temp_f+'.seizures'):
                    to_log = to_log+'sz exists.'
                    temp_annotation = wfdb.rdann(temp_f, 'seizures')
                    for i in range(int(temp_annotation.sample.size/2)):
                        temp_is_sz[temp_annotation.sample[i*2]:temp_annotation.sample[i*2+1]]=1
                else:
                    to_log = to_log+'No sz.'

                temp_len = temp_edf.n_times

                time_window = 8
                time_step = 4
                fs = int(1/(temp_edf.times[1]-temp_edf.times[0]))
                step_window = time_window*fs
                step = time_step*fs

                temp_is_sz_ind = np.array(
                    [temp_is_sz[i*step:i*step+step_window].sum()/step_window for i in range((temp_len-step_window)//step)]
                )
                del temp_is_sz

                temp_0_sample_size = round(p*np.where(temp_is_sz_ind==0)[0].size)
                temp_1_sample_size = np.where(temp_is_sz_ind>0)[0].size

                # sz data
                temp_ind = list(np.where(temp_is_sz_ind>0)[0])
                for i in temp_ind:
                    array_signals[counter, :, :] = temp_signals[:, i*step:i*step+step_window]
                    array_is_sz[counter] = True
                    counter = counter+1

                # no sz data
                temp_ind = random.sample(list(np.where(temp_is_sz_ind==0)[0]), temp_0_sample_size)
                for i in temp_ind:
                    array_signals[counter, :, :] = temp_signals[:, i*step:i*step+step_window]
                    array_is_sz[counter] = False
                    counter = counter+1

                to_log += '{} signals added: {} w/o sz, {} w/ sz.'.format(
                    temp_0_sample_size+temp_1_sample_size, temp_0_sample_size, temp_1_sample_size
                )
                print("array_signals",array_signals.shape, array_signals[:2], "temp_signals",temp_signals.shape, temp_signals[:2])

            else:
                to_log += 'Not appropriate channel labels. Reading skipped.'.format(n)


            temp_edf.close()

        np.save('signal_samples', array_signals)
        np.save('is_sz', array_is_sz)


In [ ]:
# same as above but with comments
def extract_edf2(folder, n_samples, n_channel_labels, window_size):
    if os.path.exists(" ") and os.path.exists(" "):
        print("File  placeholder name and seizure_labels.npy exists.")
        return
    signals_np = np.zeros((n_samples, n_channel_labels, window_size), dtype=np.float32)
    labels_np = np.zeros(n_samples, dtype=bool)


    # Read  edf, rename file channel to match names from channel labels list. When files have multiple channel names, only first is picked.
    for number, file in enumerate(tqdm.tqdm(folder)):
        log = f"Reading file {number} "
        edf_file = mne.io.read_raw_edf(file, preload=False)

        n_label_match = sum([any([0 if re.match(ch, ch_name) is None else 1 for ch_name in edf_file.ch_names]) for ch in channel_labels])
        if n_label_match == len(channel_labels):
            dict_ch_name = {sorted([ch_name for ch_name in edf_file.ch_names if re.match(ch, ch_name) is not None])[0]: ch for ch in channel_labels}
            edf_file.rename_channels(dict_ch_name)

            # Retrieve EEG (in microvolts) ,  annotations
            has_seizure = np.zeros((edf_file.n_times,))
            signals_ = edf_file.get_data(picks=channel_labels) * TO_MICROVOLTS

            if os.path.exists(file + '.seizures'):
                log += "positive seizure"
                has_annotation = wfdb.rdann(file, 'seizures')
                for idx in range(int(has_annotation.sample.size / 2)):
                    has_seizure[has_annotation.sample[idx * 2]:has_annotation.sample[idx * 2 + 1]] = 1
            else:
                log += "negative seizure"

            # create seizure index, and calculate proportion of signal which shows seizures
            sampling_freq = int(1/(edf_file.times[1] - edf_file.times[0]))
            print("sampling frequency ",sampling_freq)
            window_stride = int(sampling_freq * STEP_TIME)
            print(f"Window stride: {window_stride}, Window size: {window_size}")
            is_seizure_idx = np.array([has_seizure[idx * window_stride:idx * window_stride + window_size].sum() / window_size for idx in range((edf_file.n_times - window_size) // window_stride)])

            # populate numpy array with EEG , annotation data
            noseizure_n_size = round(SEIZURE_PROPORTION * np.where(is_seizure_idx == 0)[0].size)
            seizure_n_size = np.where(is_seizure_idx > 0)[0].size


            # non seizure data are by far larger than seizure data. To avoid overfitting, and bias, non seizure data is randomly subsampled. This prevents model from being overwhelmed by large non seizure data
            count = 0
            # no seizure
            temp_negative = random.sample(list(np.where(is_seizure_idx == 0)[0]), noseizure_n_size)
            for value in temp_negative:
                start_idx = value * window_stride
                end_idx = start_idx + window_size
                segment = signals_[:, start_idx:end_idx]
                print(f"Segment shape: {segment.shape}")  # Debug print
                if segment.shape == (n_channel_labels, window_size):
                    signals_np[count, :, :] = segment
                    labels_np[count] = False
                    count += 1
                else:
                    print(f"Skipping segment due to shape mismatch: {segment.shape}")

            # seizure
            temp_positive = list(np.where(is_seizure_idx > 0)[0])
            for value in temp_positive:
                start_idx = value * window_stride
                end_idx = start_idx + window_size
                segment = signals_[:, start_idx:end_idx]
                print(f"Segment shape: {segment.shape}")  # Debug print
                if segment.shape == (n_channel_labels, window_size):
                    signals_np[count, :, :] = segment
                    labels_np[count] = True
                    count += 1
                else:
                    print(f"Skipping segment due to shape mismatch: {segment.shape}")

            print(f"{noseizure_n_size + seizure_n_size} EEG signals added {seizure_n_size} with, {noseizure_n_size} without seizures")
        else:
            print(f"Unable to read {file}")

        edf_file.close()

    np.save('eeg_signals', signals_np)
    np.save('seizure_labels', labels_np)

In [ ]:
count1, ch_labels1, step_window1 = read_edf(train_files)
extract_edf2(train_files, count1, ch_labels1, step_window1)

### Recurrent neural network ( Bidirectional LSTM )

In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Dropout, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

class RnnModel:
    def __init__(self, train_signals, train_labels, test_signals, test_labels):
        self.train_signals = train_signals
        self.train_labels = train_labels
        self.test_signals = test_signals
        self.test_labels = test_labels
        self.model = None
        self.recall = None

    def train(self, save_filepath='best_model.keras'):
        """Train the model and save the best one during training."""
        # Instantiate sequential API
        model = Sequential()

        timesteps = self.train_signals.shape[1]
        features = self.train_signals.shape[2]

        # Add first Bidirectional LSTM layer
        model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(timesteps, features)))
        # Add dropout to prevent overfitting
        model.add(Dropout(0.3))

        # Add second Bidirectional LSTM layer
        model.add(Bidirectional(LSTM(64, return_sequences=True)))
        # Add dropout to prevent overfitting
        model.add(Dropout(0.3))

        # Add third Bidirectional LSTM layer
        model.add(Bidirectional(LSTM(32)))

        # Output layer with 1 neuron and sigmoid activation for binary classification
        model.add(Dense(1, activation='sigmoid'))

        # Compile the model with binary cross-entropy loss and Adam optimizer
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['Recall', 'Precision'])

        # Define the ModelCheckpoint callback to save the best model
        checkpoint_callback = ModelCheckpoint(save_filepath, save_best_only=True, monitor='val_loss', mode='min')

        # Train the model
        history = model.fit(self.train_signals, self.train_labels, epochs=10, batch_size=64,
                            validation_split=0.3, callbacks=[checkpoint_callback])

        self.model = model
        self.recall = history.history['val_recall']

    def evaluate(self):
        if self.model is None:
            print("Model has not been trained yet.")
            return

        loss, recall, precision = self.model.evaluate(self.test_signals, self.test_labels)
        print(f"Test loss: {loss:.4f}, Test Recall: {recall:.4f}, Test Precision: {precision:.4f}")

        # Prediction
        y_prediction = (self.model.predict(self.test_signals) > 0.5).astype(int)
        y_true = self.test_labels

        # Plot confusion matrix
        cm = confusion_matrix(y_true, y_prediction)
        cm_labels = ["Non-seizure", "Seizure"]
        ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=cm_labels).plot(cmap=plt.cm.Blues)
        plt.show()

        # Plot recall vs. epochs graph
        epochs = range(1, len(self.recall) + 1)
        plt.plot(epochs, self.recall)
        plt.ylabel('Recall')
        plt.xlabel('Epochs')
        plt.legend(['Train', 'Test'], loc='upper left')
        plt.show()

    @staticmethod
    def load_model_from_file(filename):
        """Load a pre-trained model from a file and return it."""
        try:
            model = load_model(filename)
            print(f"Model loaded from {filename}")
            return model
        except Exception as e:
            print(f"Error loading model: {e}")
            return None



In [ ]:
# Initialize and train the model
rnn_model = RnnModel(train_signals, train_labels, test_signals, test_labels)
rnn_model.train(save_filepath='best_rnn_model.keras')

# Later, load the best model into a new instance
loaded_model = RnnModel.load_model_from_file('best_rnn_model.keras')

# You can then evaluate the loaded model without retraining
if loaded_model:
    # Create a new RnnModel instance and set its model to the loaded one
    new_rnn_model = RnnModel(train_signals, train_labels, test_signals, test_labels)
    new_rnn_model.model = loaded_model
    new_rnn_model.evaluate()


### convolutional lstm

In [ ]:
#delete
class ConvLSTM:
    def __init__(self, shape):
        self.shape = shape

    def hybrid(self):
        model = Sequential()
        model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=self.shape))
        model.add(MaxPooling1D(pool_size=2))

        model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
        model.add(MaxPooling1D(pool_size=2))

        #model.add(Flatten())
        model.add(LSTM(128, activation='relu', return_sequences=False))

        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Recall','Precision'])
        model.summary()
        return model


### Prototype model convolutional lstm

In [ ]:
class PrototypeModel:
    def __init__(self, shape):
        self.shape = shape

    def hybrid(self):
        model = Sequential()
        model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=self.shape))
        model.add(MaxPooling1D(pool_size=2))

        model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
        model.add(MaxPooling1D(pool_size=2))

        #model.add(Flatten())
        model.add(LSTM(128, activation='relu', return_sequences=False))

        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Recall','Precision'])
        model.summary()
        return model


In [ ]:
p_model = PrototypeModel(train_signals.shape[1:])
cnn_lstm = p_model.hybrid()

In [ ]:
callbacks = [ModelCheckpoint('seizure_CNN_LSTM.keras', save_best_only=True)]
cnn_lstm_history = cnn_lstm.fit(train_signals, train_labels, epochs=10, batch_size=32,
            validation_data=(val_signals,val_labels), callbacks=callbacks)

#### Plot graph
if util.find_spec('matplotlib') is None:
    print("matplotlib module not available")

In [ ]:
import matplotlib.pyplot as plt
def plot_summary(history):
    epochs = range(1,len(history.history['loss'])+1)
    recall = history.history['recall']
    precision = history.history['precision']
    plt.figure()
    plt.plot(epochs, recall, 'bo', label='Recall')
    plt.plot(epochs, precision, 'g+', label='Precision')
    plt.title('Recall and Precision')
    plt.legend()
    plt.show()

r_= cnn_lstm_history.history['recall']
p_ = cnn_lstm_history.history['precision']

convolutional transformer

In [ ]:
# Transformer encoder layer
class Transformer(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(Transformer, self).__init__()
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.sequential = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim),
        ])
        self.normalised1 = LayerNormalization(epsilon=1e-6)
        self.normalised2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.attention(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.normalised1(inputs + attn_output)
        ffn_output = self.sequential(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.normalised2(out1 + ffn_output)

class CNN_Transformer:
    def __init__(self, input_shape, num_filters=32, kernel_size=3, embed_dim=64, num_heads=4, ff_dim=128, num_classes=1):
        self.input_shape = input_shape
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.num_classes = num_classes
        self.model = self.build_model()

    def build_model(self):
        inputs = Input(shape=self.input_shape)

        # CNN Block: Extract spatial-temporal features
        x = Conv1D(filters=self.num_filters, kernel_size=self.kernel_size, activation="relu", padding="same")(inputs)
        x = Conv1D(filters=self.num_filters * 2, kernel_size=self.kernel_size, activation="relu", padding="same")(x)
        x = MaxPooling1D(pool_size=2)(x)
        x = BatchNormalization()(x)

        # Additional CNN layers for deeper feature extraction
        x = Conv1D(filters=self.num_filters * 4, kernel_size=self.kernel_size, activation="relu", padding="same")(x)
        x = MaxPooling1D(pool_size=2)(x)
        x = BatchNormalization()(x)

        # Flatten for embedding
        x = Dense(self.embed_dim)(x)

        # Transformer Block
        x = Transformer(embed_dim=self.embed_dim, num_heads=self.num_heads, ff_dim=self.ff_dim)(x)

        # Global pooling
        x = GlobalAveragePooling1D()(x)

        # Fully connected layers for classification
        x = Dense(64, activation="relu")(x)
        x = Dropout(0.3)(x)
        outputs = Dense(self.num_classes, activation="sigmoid")(x)

        model = Model(inputs=inputs, outputs=outputs)
        return model

    def compile(self):
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    def fit(self, X_train, y_train, batch_size=32, epochs=10, validation_data=None):
        return self.model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=validation_data)

    def evaluate(self, X_test, y_test):
        return self.model.evaluate(X_test, y_test)


pure transformer

In [ ]:
# input_shape = (samples,23, 2560)  # (samples, channels, time steps)

# Transformer encoder layer
class Transformer(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(Transformer, self).__init__()
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        # A feed forward network with non linear transformation of features at each neuron
        self.sequential = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            # Each channel is transformed to a dense representation using Dense(embed_dim).
            Dense(embed_dim),
        ])
        self.normalised1 = LayerNormalization(epsilon=1e-6)
        self.normalised2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.attention(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.normalised1(inputs + attn_output)
        ffn_output = self.sequential(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.normalised2(out1 + ffn_output)

# Build the model
def build_model(input_shape, num_heads=4, ff_dim=128, embed_dim=64, num_classes=1):
    inputs = Input(shape=input_shape)

    # Flatten the time series for embedding
    x = Dense(embed_dim)(inputs)  # Create embedding
    x = Transformer(embed_dim, num_heads, ff_dim)(x)

    # Global pooling
    x = GlobalAveragePooling1D()(x)

    # Fully connected output
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation="sigmoid")(x)  # For binary classification

    model = Model(inputs=inputs, outputs=outputs)
    return model
